In [13]:
# ==================================================
# Paso 1: Librerías que vamos a necesitar
# ==================================================

import pandas as pd
import numpy as np
import requests #Para poder obtener los datos de intenet.
import logging #Para el manejo de información y errores.

#Configuración básica del logging de nivel mínimo INFO.
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

# ==================================================
# Paso 2: Descargar datos desde una URL
# ==================================================

url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"

try:
    logging.info("Descargando datos de la URL...")

    respuesta = requests.get(url) #solicitamos los datos a la url
    respuesta.raise_for_status() #lanza un error si hubo algún problema con la conexión con la url y por lo tanto no se realizó la descarga.

    #Guardamos el contenido descargado en la carpeta RAW_DATA
    with open("../DATA/RAW_DATA/datos_covid.csv", "wb") as carpeta:
        carpeta.write(respuesta.content)

    logging.info("Datos descargados y guardados en RAW_DATA") #mensaje para confirmar que los datos fueron descargados y guardados en la carpeta
    
except requests.exceptions.RequestException as e: #error si no se pudieron descargar los datos.
    logging.error("Error al descargar los datos:", e)

# ==================================================
# Paso 3: Cargar CSV en pandas
# ==================================================

try:
    df = pd.read_csv("../DATA/RAW_DATA/datos_covid.csv")
    logging.info(f"Dataset cargado con {df.shape[0]} filas y {df.shape[1]} columnas")
except Exception as e:
    logging.error(f"No se pudo cargar el dataset: {e}")

# ==================================================
# Paso 4: Limpieza y transformación
# ==================================================

try:
    df.rename(columns = {"Province/State":"Provincia","Country/Region":"País"}, inplace = True) #renombramos el nombre de dos columnas
    #Ahora vamos a sumar cuantos casos han habido para cada pais y el total lo mostramos en una columna nueva.
    df["Casos_totales"] = df.iloc[:,4:].sum(axis=1)

    logging.info("¡Transformaciones realizadas!")
except Exception as e:
    logging.error(f"Error durante la transformación de los datos: {e}")

# ==================================================
# Paso 5: Guardar dataset limpio
# ==================================================

try:
    df.to_csv("../DATA/CLEAN_DATA/datos_covid_transformados.csv", index = False)
    df.to_parquet("../DATA/CLEAN_DATA/datos_covid_transformados.parquet", engine = "pyarrow", index = False)
    logging.info("¡Datasets transformados guardados correctamente en formato csv y parquet!")
except Exception as e:
    logging.error(f"Error al guardar el dataset transformado en formato cvs y parque: {e}")
    


INFO: Descargando datos de la URL...
INFO: Datos descargados y guardados en RAW_DATA
INFO: Dataset cargado con 289 filas y 1147 columnas
INFO: ¡Transformaciones realizadas!
INFO: ¡Datasets transformados guardados correctamente en formato csv y parquet!


In [12]:
df.head()

,Provincia,País,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23,Casos_totales
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209340,209358,209362,209369,209390,209406,209436,209451,209451,129988469
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334408,334408,334427,334427,334427,334427,334427,334443,334457,185562654
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271448,271463,271469,271469,271477,271477,271490,271494,271496,182741650
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47875,47875,47875,47875,47875,47875,47875,47890,47890,24547525
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105277,105277,105277,105277,105277,105277,105277,105288,105288,60025203
